In [25]:
import pandas as pd;
import random;

OBJECTS/VARIABLES:

Antibody - list of Numerical values (Reals?) , represented by a row of a dataframe?
Population - List[Antibodies], represented by the total dataframe?

Target Size (i.e number of antibodies to be created) = (Size of Majority Class) - (Size of minority class)



FUNCTIONS:

Initializing :- input: original dataframe

                         do: Get bounds of minority class by taking the highest and lowest values in each of the [n] dimensions

                         How: Do we take the whole minority class? or do we sample part or parts of it to generate our bounds. 

                         output: Upper and lower bounds of the minority class


Creation :- Input: Bounds of the min class

        do: Create a set of antibodies

        input: minority Dataframe 

        How:
            Possibilities:
            (As the Malhanabois paper does it) Take a random value between the bounds of the minority class feature as the datapoint

            (Nikhil Just sample minority class based on imbalance rate (doesn't require bounds)

            (Adam) Take a random value, as in the paper's method, but off of a weighted curve? as in we could randomize, 
                but add some preference for values close to the boundary, or close to the center, etc.
                        -We could set this as a parameter, bell curve, linear. This same function could be a parameter in the mutation stage.
                        -If the density within the bounds is concentrated on one side, add bias towards that side in the random value;
        
        Challenges: 
            How do we deal with the different data categories (e.g nominal, ordinal, and continuous)? Continuous is easy, just a number in a range. Ordinal is ???, nominal is difficult, even if one-hot encoded, we might random to have two values that should be exclusive (e.g an item being both blue and red). How do other imputation algorithms work with these problems? DO they even work with these problems?
        output: Initial Population as a DF?



**Initialization**

In [45]:
df = pd.read_csv("./Data/TitanicData_syn.csv")
df= df.drop("Result", axis=1)
df

,class2,AGE,Sex
0,0.0214,-0.228,-1.920
1,0.9650,-0.228,0.521
2,0.0214,-0.228,0.521
3,0.9650,-0.228,0.521
4,0.9650,-0.228,0.521
...,...,...,...
1996,-0.9230,-0.228,-1.920
1997,0.0214,-0.228,0.521
1998,0.0214,4.380,-1.920
1999,-0.9230,-0.228,0.521


In [49]:

def get_bounds(minorityDF) -> tuple:
    out = []
    for col in minorityDF:
        colMax = df[col].max()
        colMin = df[col].min()
        out += [(col, colMin, colMax, )]
    return out

#This only works for continuous values. We will have to code a version for binary fields (We assume any categorical columns have been encoded)
def Creation(minorityDF, totalPopulation : int, weightingFunction : str = "Default"): 
    population = [] #Initializing the empty population
    
    for i in range(totalPopulation): #For every antibody to be created

        antibody = [] #Initializing a single antibody

        for col in get_bounds(minorityDF): #Iterate through the columns/dimensions/labels of the minority class for each antibody 

            antibody += [round(random.uniform(col[1],col[2]),4)] #Add a random value between the lower and upper bounds to the antibody

        population+=[antibody] #add the created antibody to the population

    return population

print(Creation(df, 3))

('class2', -1.87, 0.965)
('AGE', -0.228, 4.38)
('Sex', -1.92, 0.521)
('class2', -1.87, 0.965)
('AGE', -0.228, 4.38)
('Sex', -1.92, 0.521)
('class2', -1.87, 0.965)
('AGE', -0.228, 4.38)
('Sex', -1.92, 0.521)
[[0.3271, 0.1324, -0.9055], [-0.8775, 2.9561, -0.3356], [-0.4724, 1.8725, -0.0124]]
